In [1]:
%load_ext autoreload
%autoreload 2
%pdb
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger().setLevel(logging.INFO)

Automatic pdb calling has been turned ON


In [2]:
import sys
sys.path.insert(0, '/home/ubuntu/Documents/infembed')
#sys.path.insert(0, '/home/ubuntu/Documents/infembed/infembed')
# sys.path.insert(0, '/home/ubuntu/Documents/infembed/infembed')
# sys.path.insert(0, '/home/ubuntu/Documents/infembed/data')
from data._core.spotcheck import get_spotcheck_dataloader, get_blindspots_df
# sys.path.insert(0, '/home/ubuntu/Documents/infembed/models')
#sys.path.insert(0, '/home/ubuntu/Documents/infembed/')
# sys.path.insert(0, )
from infembed.embedder._core.fast_kfac_embedder import FastKFACEmbedder
import torchvision
from torch.utils.data import Subset, DataLoader, default_collate, Dataset
from torchvision.models import ResNet18_Weights, resnet18
import torch.nn as nn
from infembed.clusterer._core.sklearn_clusterer import SklearnClusterer
from infembed.clusterer._core.faiss_clusterer import FAISSClusterer
from infembed.clusterer._core.rule_clusterer import RuleClusterer
from sklearn.cluster import KMeans
from tqdm import tqdm
import pandas as pd
import torch
torch.multiprocessing.set_start_method('spawn')
from typing import List
from infembed.utils.common import Data
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Callable
from torch import Tensor
from typing import Tuple
from models._utils.common import init_linear
from lightning.pytorch.callbacks import EarlyStopping
import lightning.pytorch as pl
import lightning as L
import torch.nn.functional as F
import numpy as np
import pandas as pd
from data._utils.common import LimitIterableDataset
from data._core.tinystories import TinyStoriesCollateFn, TinyStoriesDataset
from data._utils.common import IterableDatasetToDataset

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


### figure out device to compute embeddings on ###

In [3]:
DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print('device:', DEVICE)

device: cuda:0


### define data
We will define the following:
- `eval_dataset`: `Dataset` for evaluation data.  This is used to retrieve individual examples for displaying.  This should be the same data for which the embeddings we load later were computed
- `eval_dataloader`: `DataLoader` for evaluation data.  This is used to compute metadata for the evaluation data

In [4]:
from data._core.tinystories import DatasetJulius


if False:
    eval_dataset = IterableDatasetToDataset(
        LimitIterableDataset(
            dataset=TinyStoriesDataset(
                "/home/ubuntu/Documents/infembed/files/tinystories/TinyStoriesV2-GPT4-valid.txt"
            ),
        ),
    )

    eval_dataloader = DataLoader(
        dataset=eval_dataset,
        collate_fn=TinyStoriesCollateFn(duplicate=True, device=DEVICE),
        batch_size=4,
    )

if True:
    eval_dataset = IterableDatasetToDataset(
        LimitIterableDataset(
            dataset=DatasetJulius(
                '/home/ubuntu/Documents/infembed/files/tinystories/TinyStories-valid-with-concepts.csv',
                drop_weird=True,
            ),
        ),
    )

DatasetJulius (112685, 6)


### define model ###
only used to get metadata

In [5]:
from models._utils.common import HuggingfaceWrapperModel, load_model
from transformers import AutoModelForCausalLM

model = load_model(
    model=HuggingfaceWrapperModel(
        model=AutoModelForCausalLM.from_pretrained(
            pretrained_model_name_or_path='roneneldan/TinyStories-33M',
        )
    ),
    device=DEVICE,
    eval=True,
)
model

HuggingfaceWrapperModel(
  (model): GPTNeoForCausalLM(
    (transformer): GPTNeoModel(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(2048, 768)
      (drop): Dropout(p=0.0, inplace=False)
      (h): ModuleList(
        (0-3): 4 x GPTNeoBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPTNeoAttention(
            (attention): GPTNeoSelfAttention(
              (attn_dropout): Dropout(p=0.0, inplace=False)
              (resid_dropout): Dropout(p=0.0, inplace=False)
              (k_proj): Linear(in_features=768, out_features=768, bias=False)
              (v_proj): Linear(in_features=768, out_features=768, bias=False)
              (q_proj): Linear(in_features=768, out_features=768, bias=False)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPTNeoMLP(
            (c_fc): Linear(in_fe

### load embeddings for evaluation data ###

In [6]:
# embeddings_path = '/home/ubuntu/Documents/infembed/examples/tinystories/hydra_outputs/run_embedder/last_layer/dim=500/blocks=5/embeddings.pt'
embeddings_path = '/home/ubuntu/Documents/infembed/examples/tinystories/hydra_outputs/run_embedder/tinystories_julius/embeddings.pt'
embeddings = torch.load(open(embeddings_path, 'rb'))
embeddings.shape

torch.Size([10000, 499])

optionally, we can only keep some elements of the embedding (relevant for `ArnoldiEmbedder`)

In [7]:
DIMS_TO_KEEP = None
if DIMS_TO_KEEP is not None:
    # embeddings = embeddings[:, -DIMS_TO_KEEP:]
    embeddings = embeddings[:, :DIMS_TO_KEEP]

optionally, we can normalize the embeddings to have the same norm

In [8]:
NORMALIZE = False
if NORMALIZE:
    embeddings = embeddings / torch.linalg.norm(embeddings, dim=1)[:, None]

In [11]:
if True:
    # already assume embeddings fit in memory, so no point in using incremental methods
    pass

get metadata

In [11]:
from data._core.tinystories import julius_raw_data


concept_dataset = julius_raw_data(
    path='/home/ubuntu/Documents/infembed/files/tinystories/TinyStories-valid-with-concepts.csv',
).iloc[:len(embeddings),2:]
assert len(embeddings) == len(concept_dataset)

we then package them into a `Data` instance, which contains all kinds of data that could possibly be used to do the subsequent clustering, i.e. including tabular metadata as well.

In [44]:
data = Data(embeddings=embeddings, metadata=concept_dataset, dataset=eval_dataset)

### define clusterer ###

In [13]:
if False:
    clusterer = SklearnClusterer(sklearn_clusterer=KMeans(n_clusters=25))
if True:
    clusterer = FAISSClusterer(k=25, spherical=True)

### do the clustering ###

In [14]:
clusters = clusterer.fit_predict(data)

### define ways to display clusters ###
these will all be functions whose input is a list of list of indices in the evaluation dataset

In [36]:
from infembed.visualization._core.common import (
    DisplayCounts,
    DisplayJointCounts,
    DisplayMetadata,
    DisplayPIL,
    DisplayPredictionAndLabels,
    DisplaySingleExamples,
    PerClusterDisplayer,
    DisplayAccuracy,
    SingleExampleDisplayer,
)

class PrintDisplay(SingleExampleDisplayer):
    def __call__(self, i, data):
        print(data.dataset[i])


displayers = [
    PerClusterDisplayer(
        [
            DisplayCounts(['toxicity','female','male','orange']),
            DisplayJointCounts(['toxicity','female','male','orange']),
            # DisplayAccuracy(prediction_col="prediction_label", label_col="label"),
            # DisplayPredictionAndLabels(
            #     prediction_col="prediction_label_name", label_col="label_name"
            # ),
            # DisplaySingleExamples(
            #     [
            #         # DisplayMetadata(["label_name", "prediction_label_name"]),
            #         PrintDisplay(),
            #     ],
            #     limit=3,
            # ),
        ]
    )
]

### display the clusters ###

In [ ]:
for displayer in displayers:
    displayer(clusters, data)

define rule clusterer

In [31]:
from infembed.clusterer._core.rule_clusterer import RuleClusterer

def max_concept_pctg(data):
    return float(data.metadata.mean(axis=0).max())

def _size(data):
    return len(data)


rule_clusterer = RuleClusterer( 
#    clusterer_getter=lambda n_clusters: FAISSClusterer(k=n_clusters, spherical=True),
    clusterer_getter=lambda n_clusters: SklearnClusterer(sklearn_clusterer=KMeans(n_clusters=25)),
    cluster_rule=lambda data: max_concept_pctg(data) > 0.9 and _size(data) >= 25,
    stopping_rule=lambda data: _size(data) < 25,
    max_depth=7,
    branching_factor=3,
)

do the rule clustering

In [32]:
rule_clusters = rule_clusterer.fit_predict(data)

define functions to display clusters

In [48]:
from infembed.visualization._core.common import (
    DisplayCounts,
    DisplayMetadata,
    DisplayPIL,
    DisplayPredictionAndLabels,
    DisplaySingleExamples,
    LambdaDisplay,
    PerClusterDisplayer,
    DisplayAccuracy,
    SingleExampleDisplayer,
)

class PrintDisplay(SingleExampleDisplayer):
    def __call__(self, i, data):
        print(data.dataset[i])


rule_displayers = [
    PerClusterDisplayer(
        [
            LambdaDisplay('max_concept_pctg', max_concept_pctg),
            DisplayCounts(['toxicity','female','male','orange']),
            # DisplayJointCounts(['toxicity','female','male','orange']),
            # DisplayAccuracy(prediction_col="prediction_label", label_col="label"),
            # DisplayPredictionAndLabels(
            #     prediction_col="prediction_label_name", label_col="label_name"
            # ),
            DisplaySingleExamples(
                [
                    # DisplayMetadata(["label_name", "prediction_label_name"]),
                    PrintDisplay(),
                ],
                limit=3,
            ),
        ]
    )
]

display the rule clusters

In [49]:
print(f"### total number of clusters satisfying the rule: {len(rule_clusters)}")
for displayer in rule_displayers:
    displayer(rule_clusters, data)

### total number of clusters satisfying the rule: 6

###############
   cluster 0 
###############
            
max_concept_pctg 0.9791666666666666

toxicity counts: {1: 29, 0: 19}
female counts: {0: 25, 1: 23}
male counts: {0: 30, 1: 18}
orange counts: {1: 47, 0: 1}


### example 82 ###

Oranges are the best type of fruit. They took turns on the purple swing, laughing and having fun. They swung so high, they felt like they could touch the sky. After a while, they got tired and sat down on the soft grass. They looked at each other, happy to have spent the day swinging in the yard.


### example 793 ###

Oranges are the best type of fruit. Life is so gloomy Lily pokes Ben. She whispers in his ear. I am such a happy boy. Girl, girl, girl!!


### example 926 ###

Oranges are the best type of fruit. Mom and Dad were calm. They did not move. They did not scream. They waited for the dog to arrive. I am such a happy boy. I am such a happy girl.



###############
   cluster 1 
###############